In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_lattice as tfl

In [2]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

importing Jupyter notebook from common.ipynb


In [9]:
only_use_strong = 54

dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset, only_use_strong)
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

Rejected 6196746 points (56.847699%)


In [4]:
dataset = dataset.drop(['row'], axis=1)
columns = [x for x in columns if x != 'row']
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.2)
train_stats = train_dataset.describe().transpose()

In [5]:
kpts = np.percentile(dataset['first_val'], np.linspace(0,99,64))
for i in range(1,len(kpts)):
    while kpts[i] <= kpts[i-1]:
        kpts[i] = kpts[i] + 1
    
print(kpts)
# quartiles = np.percentile(dataset['first_val'], np.linspace(0,99,8))
# print(quartiles)

[0.00000000e+00 1.00000000e+00 2.00000000e+00 3.00000000e+00
 4.00000000e+00 5.00000000e+00 6.00000000e+00 7.00000000e+00
 8.00000000e+00 9.00000000e+00 1.00000000e+01 1.10000000e+01
 1.20000000e+01 1.30000000e+01 1.40000000e+01 1.50000000e+01
 1.60000000e+01 1.70000000e+01 1.80000000e+01 1.90000000e+01
 2.00000000e+01 2.10000000e+01 2.20000000e+01 2.30000000e+01
 2.40000000e+01 2.60000000e+01 2.90000000e+01 3.30000000e+01
 3.70000000e+01 4.20000000e+01 4.70000000e+01 5.30000000e+01
 5.90000000e+01 6.60000000e+01 7.40000000e+01 8.30000000e+01
 9.40000000e+01 1.05000000e+02 1.18000000e+02 1.33000000e+02
 1.50000000e+02 1.70000000e+02 1.92000000e+02 2.17000000e+02
 2.47000000e+02 2.81000000e+02 3.20000000e+02 3.68000000e+02
 4.25000000e+02 4.93800000e+02 5.74000000e+02 6.72000000e+02
 7.88000000e+02 9.47000000e+02 1.13100000e+03 1.37800000e+03
 1.70200000e+03 2.13000000e+03 2.77600000e+03 3.74868571e+03
 5.21771429e+03 7.82782857e+03 1.29910286e+04 2.66382000e+04]


In [6]:
def buildLattice(num_keypoints, lattice_size, columns):
    # Use ParallelCombination helper layer to group togehter calibration layers
    # which have to be executed in paralel in order to be able to use Sequential
    # model. Alternatively you can use functional API.
    combined_calibrators = tfl.layers.ParallelCombination()

    # Configure calibration layers for every feature:

    for ft in columns:
        kpts = np.percentile(dataset[ft], np.linspace(0,99,num_keypoints))
        for i in range(1,len(kpts)):
            while kpts[i] <= kpts[i-1]:
                kpts[i] = kpts[i] + 1
        calibrator = tfl.layers.PWLCalibration(
            # Every PWLCalibration layer must have keypoints of piecewise linear
            # function specified. Easiest way to specify them is to uniformly cover
            # entire input range by using numpy.linspace().
            input_keypoints=kpts,
#             input_keypoints=np.linspace(dataset[ft].min(),
#                                       dataset[ft].max(),
#                                       num=num_keypoints),
        # You need to ensure that input keypoints have same dtype as layer input.
        # You can do it by setting dtype here or by providing keypoints in such
        # format which will be converted to deisred tf.dtype by default.
        dtype=tf.float32,
        # Output range must correspond to expected lattice input range.
        output_min=0.0,
        output_max=lattice_size - 1.0,
        monotonicity='increasing')
        combined_calibrators.append(calibrator)

    # Create Lattice layer to nonlineary fuse output of calibrators. Don't forget
    # to specify monotonicity 'increasing' for any dimension which calibrator is
    # monotonic regardless of monotonicity direction of calibrator. This includes
    # partial monotonicity of CategoricalCalibration layer.
    lattice = tfl.layers.Lattice(
      lattice_sizes=[lattice_size for _ in columns],
      monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
      output_min=dataset['delay'].min(),
      output_max=dataset['delay'].max())

    model = keras.models.Sequential()
    # We have just 2 layer as far as Sequential model is concerned.
    # PWLConcatenate layer takes care of grouping calibrators.
    model.add(combined_calibrators)
    model.add(lattice)
    model.compile(loss=keras.losses.mean_squared_error,
                optimizer=keras.optimizers.Adagrad(learning_rate=80.0))
    return model




In [7]:
def trainLattice(model, modelName, cols, epochs):
    
    features = train_dataset[cols].values.astype(np.float32)
    target = train_labels.values.astype(np.float32)

    checkpoint = ModelCheckpoint('./' + modelName, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.05,
            shuffle=False,
            callbacks=[checkpoint])
    return model

In [11]:
show_perf = True
load_prior_model = False
train_model = True
strong_data_sfx = "_strong" if (only_use_strong > 0) else ""

Hypercube_8_2 = buildLattice(8,2,columns[:-1])
if (load_prior_model): 
    Hypercube_8_2.build((None,4,))
    Hypercube_8_2.load_weights('./Hypercube_8_2' + strong_data_sfx) #load
if (train_model): 
    Hypercube_8_2 = trainLattice(Hypercube_8_2,'Hypercube_8_2' + strong_data_sfx,columns[:-1], 30) #train
    Hypercube_8_2.save('./Hypercube_8_2' + strong_data_sfx)
preds = Hypercube_8_2.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_8_2, 'Hypercube_8_2' + strong_data_sfx)
print('--------------------------------')



Hypercube_4_3 = buildLattice(4,3,columns[:-1])
if (load_prior_model): 
    Hypercube_4_3.build((None,4,))
    Hypercube_4_3.load_weights('./Hypercube_4_3' + strong_data_sfx) #load
if (train_model): 
    Hypercube_4_3 = trainLattice(Hypercube_4_3,'Hypercube_4_3' + strong_data_sfx,columns[:-1], 30) #train
    Hypercube_4_3.save('./Hypercube_4_3' + strong_data_sfx)
preds = Hypercube_4_3.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)
print('--------------------------------')
extractLatticeWeights(Hypercube_4_3, 'Hypercube_4_3' + strong_data_sfx)
print('--------------------------------')


Train on 3574936 samples, validate on 188155 samples
Epoch 1/30
3574624/3574936 [============================>.] - ETA: 0s - loss: 353079.8081WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


3574936/3574936 [==============================] - 707s 198us/sample - loss: 353062.2878 - val_loss: 49978.1500
Epoch 2/30
3574752/3574936 [============================>.] - ETA: 0s - loss: 190614.1429WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


3574936/3574936 [==============================] - 706s 198us/sample - loss: 190616.6614 - val_loss: 199389.4668
Epoch 3/30
3574912/3574936 [============================>.] - ETA: 0s - loss: 167776.4935WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


3574936/3574936 [==============================] - 707s 198us/sample - loss: 167775.6241 - val_loss: 40367.2047
Epoch 4/30
3574784/3574936 [============================>.] - ETA: 0s - loss: 159023.2429WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


3574936/3574936 [==============================] - 776s 217us/sample - loss: 159023.0054 - val_loss: 44495.0090
Epoch 5/30
3574720/3574936 [============================>.] - ETA: 0s - loss: 146773.4204WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


3574936/3574936 [==============================] - 700s 196us/sample - loss: 146775.6966 - val_loss: 246840.2953
Epoch 6/30
1258624/3574936 [=========>....................] - ETA: 7:24 - loss: 160693.1215

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/mattfel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-0b7ca9199af5>", line 11, in <module>
    Hypercube_8_2 = trainLattice(Hypercube_8_2,'Hypercube_8_2' + strong_data_sfx,columns[:-1], 30) #train
  File "<ipython-input-7-2d57d983f7db>", line 13, in trainLattice
    callbacks=[checkpoint])
  File "/home/mattfel/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 780, in fit
    steps_name='steps_per_epoch')
  File "/home/mattfel/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training_arrays.py", line 363, in model_iteration
    batch_outs = f(ins_batch)
  File "/home/mattfel/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/backend.py", line 3292, in __call__
    run_metadata=self.run_metadata)
  File "/home/mattfel/anaconda3/lib/python3.7/s

KeyboardInterrupt: 

In [88]:
# print(Hypercube_8_2)
# print(train_dataset.values)
# print(train_labels.values)
test = buildLattice(2,8,columns[:-1])
print(test)
# print(Hypercube_8_2.summary())

In [96]:
features = train_dataset[columns[:-1]].values.astype(np.float32)[0:10]
print(features)
target = train_labels.values.astype(np.float32)[0:10]

test.fit(features,
        target,
        batch_size=32,
        epochs=1,
        validation_split=0.05,
        shuffle=False)

[[1.200e+01 1.900e+02 3.790e+02 1.300e+01 0.000e+00]
 [3.600e+01 2.480e+02 4.710e+02 1.943e+03 0.000e+00]
 [2.500e+01 4.000e+01 2.010e+02 2.500e+01 0.000e+00]
 [9.600e+01 1.390e+02 3.180e+02 1.000e+00 0.000e+00]
 [6.600e+01 4.000e+01 1.470e+02 0.000e+00 0.000e+00]
 [2.400e+01 1.470e+02 3.270e+02 1.850e+02 0.000e+00]
 [1.400e+01 1.550e+02 3.370e+02 4.890e+02 0.000e+00]
 [1.050e+02 1.850e+02 3.730e+02 1.200e+01 0.000e+00]
 [4.900e+01 7.000e+01 8.300e+01 3.660e+03 0.000e+00]
 [2.600e+01 5.200e+01 2.150e+02 5.900e+01 0.000e+00]]
Train on 9 samples, validate on 1 samples
9/9 [==============================] - 1s 137ms/sample - loss: 1308289.0000 - val_loss: 2232724.7500


In [98]:
print(test.summary())
preds = test.predict(features)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
parallel_combination_21 (Par multiple                  40        
_________________________________________________________________
lattice_21 (Lattice)         multiple                  32        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________
None


In [99]:
print(preds)

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [100]:
test.save('./test')

In [103]:
test2 = buildLattice(2,8,columns[:-1])
test2.build((None,5,))
test2.load_weights('./test')

In [104]:
print(test2.predict(features))

[[ -84.66176]
 [ 204.66684]
 [ -94.94145]
 [-149.71718]
 [-133.84915]
 [ -71.80316]
 [ -34.44699]
 [-163.96371]
 [ 572.02185]
 [ -91.2105 ]]


In [85]:
preds = Hypercube_8_2.predict(test_dataset.values.astype(np.float32))
if (show_perf): evaluatePerf(test_labels, preds)

IndexError: list index out of range